In [6]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pycep_correios
import geopy
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

TypeError: function() argument 'code' must be code, not str

In [10]:
from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# conexão com o banco de dados SQL Server
engine = create_engine(os.environ.get('DATABASE_URL_SQL_SERVER'), echo=True)

In [3]:

cep = '03987-110'
endereco = pycep_correios.get_address_from_cep(cep, webservice=pycep_correios.WebService.APICEP)
print(endereco)

{'bairro': 'Vila Portuguesa', 'cep': '03987-110', 'cidade': 'São Paulo', 'logradouro': 'Rua dos Espigueiros', 'uf': 'SP', 'complemento': ''}


In [7]:
geolocator = Nominatim(user_agent="test_app")
location = geolocator.geocode(endereco['logradouro'] + ", " + endereco['cidade'])
print(location.latitude, location.longitude)

NameError: name 'Nominatim' is not defined